In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
application_record = pd.read_csv('application_record.csv')
credit_record = pd.read_csv('credit_record.csv')

# Inspecting 'application_record' dataset

In [ ]:
application_record.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [ ]:
application_record = application_record.rename(columns={
  'CODE_GENDER':'gender',
  'FLAG_OWN_CAR':'owns_car',
  'FLAG_OWN_REALTY':'owns_property',
  'CNT_CHILDREN':'n_children',
  'AMT_INCOME_TOTAL':'annual_income',
  'NAME_INCOME_TYPE':'income_type',
  'NAME_EDUCATION_TYPE':'education_type',
  'NAME_FAMILY_STATUS':'marital_status',
  'NAME_HOUSING_TYPE':'housing_type',
  'DAYS_BIRTH': 'days_birth',
  'DAYS_EMPLOYED':'days_employed',
  'FLAG_EMAIL':'has_email',
  'FLAG_MOBIL': 'has_mobile',
  'FLAG_WORK_PHONE':'has_work_phone',
  'FLAG_PHONE':'has_phone',
  'OCCUPATION_TYPE':'occupation',
  'CNT_FAM_MEMBERS':'n_fam_members'})

## Check for duplication

In [ ]:
application_record.duplicated().sum()

0

## Check for null values

In [ ]:
application_record.isnull().sum()

,0
ID,0
gender,0
owns_car,0
owns_property,0
n_children,0
annual_income,0
income_type,0
education_type,0
marital_status,0
housing_type,0


## Converting 'days_birth' to 'age' and 'days_employed' to 'years_employed'

In [ ]:
application_record['years_employed'] = application_record['days_employed'] / -365
application_record['age'] = application_record['days_birth'] / -365

In [ ]:
application_record.drop(columns=['days_birth', 'days_employed'], inplace=True)

## Fill in missing values of 'occupation' column

In [ ]:
# change all years_employed < 0 to 0
application_record.loc[application_record['years_employed'] < 0, 'years_employed'] = 0

# Fill missing values in occupation if years_employed > 0 to 'other'
application_record.loc[application_record['years_employed'] > 0, 'occupation'] = application_record.loc[application_record['years_employed'] > 0, 'occupation'].fillna('other')

# Fill missing values in occupation if years_employed = 0 to 'unemployed'
application_record.loc[application_record['years_employed'] == 0, 'occupation'] = application_record.loc[application_record['years_employed'] == 0, 'occupation'].fillna('unemployed')

# Update occupation to "retired" for unemployed males over 60 and females over 50
application_record.loc[
    (application_record['occupation'] == 'unemployed') &
    (
        ((application_record['gender'] == 'M') & (application_record['age'] >= 60)) |  # Males over 60
        ((application_record['gender'] == 'F') & (application_record['age'] >= 50))    # Females over 50
    ),
    'occupation'
] = 'retired'

In [ ]:
application_record['occupation'].value_counts()

,count
occupation,
Laborers,78240
retired,68408
other,58874
Core staff,43007
Sales staff,41098
Managers,35487
Drivers,26090
High skill tech staff,17289
Accountants,15985


## Checking for non-logical instances in `n_children` and `n_fam_members`

In [ ]:
print(application_record[['n_children', 'n_fam_members']][application_record['n_children'] > application_record['n_fam_members']])

# remove rows where n_children > n_fam_members
application_record = application_record[application_record['n_children'] <= application_record['n_fam_members']]

        n_children  n_fam_members
434494           2            1.0
434543           2            1.0
434960           2            1.0
435100           2            1.0
435108           2            1.0
435110           2            1.0
435135           2            1.0
435182           2            1.0
435331           2            1.0
435335           2            1.0
435363           2            1.0
435426           2            1.0
435471           2            1.0
435499           2            1.0
435536           2            1.0
435597           2            1.0
435664           2            1.0
435851           2            1.0
435925           2            1.0
436006           2            1.0
436039           2            1.0
436483           2            1.0
436495           2            1.0
436562           2            1.0
436621           2            1.0
436642           2            1.0
436832           3            1.0
436908           2            1.0
437094        

## Convert binary categorical features to numerical features

In [ ]:
# owns_car and owns_property Y to 1 and N to 0
application_record['owns_car'] = application_record['owns_car'].map({'Y': 1, 'N': 0})
application_record['owns_property'] = application_record['owns_property'].map({'Y': 1, 'N': 0})
# gender M to 1 and F to 0
application_record['gender'] = application_record['gender'].map({'M': 1, 'F': 0})

# Inspecting 'credit_record'

In [ ]:
credit_record.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


## Check for duplication

In [ ]:
credit_record.duplicated().sum()

0

## Check for null values

In [ ]:
credit_record.isnull().sum()

,0
ID,0
MONTHS_BALANCE,0
STATUS,0


In [ ]:
credit_record['STATUS'].value_counts()

,count
STATUS,
C,442031
0,383120
X,209230
1,11090
5,1693
2,868
3,320
4,223


# Labeling on 'credit_record'

In [ ]:
# Create a new column called good debt, map status = x, c, 0 to 1 and others to 0
credit_record['good_debt'] = credit_record['STATUS'].map({'X': 1, 'C': 1, '0': 1})

# create a new column for bad debt, map status = 2, 3, 4, 5 to 1 and others to 0
credit_record['bad_debt'] = credit_record['STATUS'].map({'1': 1, '2': 1, '3': 1, '4': 1, '5': 1}).fillna(0)

print(credit_record)

              ID  MONTHS_BALANCE STATUS  good_debt  bad_debt
0        5001711               0      X        1.0       0.0
1        5001711              -1      0        1.0       0.0
2        5001711              -2      0        1.0       0.0
3        5001711              -3      0        1.0       0.0
4        5001712               0      C        1.0       0.0
...          ...             ...    ...        ...       ...
1048570  5150487             -25      C        1.0       0.0
1048571  5150487             -26      C        1.0       0.0
1048572  5150487             -27      C        1.0       0.0
1048573  5150487             -28      C        1.0       0.0
1048574  5150487             -29      C        1.0       0.0

[1048575 rows x 5 columns]


In [ ]:
# group by id, compare total of good debt and bad debt, create 'label' column if good debt > bad debt then 0 else 1
credit_record_data_grouped = credit_record.groupby('ID').agg({'good_debt': 'sum', 'bad_debt': 'sum'}).reset_index()
credit_record_data_grouped['label'] = np.where(credit_record_data_grouped['good_debt'] > credit_record_data_grouped['bad_debt'], 1, 0)

print(credit_record_data_grouped)

            ID  good_debt  bad_debt  label
0      5001711        4.0       0.0      1
1      5001712       19.0       0.0      1
2      5001713       22.0       0.0      1
3      5001714       15.0       0.0      1
4      5001715       60.0       0.0      1
...        ...        ...       ...    ...
45980  5150482       18.0       0.0      1
45981  5150483       18.0       0.0      1
45982  5150484       13.0       0.0      1
45983  5150485        2.0       0.0      1
45984  5150487       30.0       0.0      1

[45985 rows x 4 columns]


# Merging 'application_record' with 'label'

In [ ]:
merged_data = pd.merge(application_record, credit_record_data_grouped, on='ID', how='inner')

In [ ]:
merged_data.head()

,ID,gender,owns_car,owns_property,n_children,annual_income,income_type,education_type,marital_status,housing_type,...,has_work_phone,has_phone,has_email,occupation,n_fam_members,years_employed,age,good_debt,bad_debt,label
0,5008804,1,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,0,0,other,2.0,12.443836,32.890411,15.0,1.0,1
1,5008805,1,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,1,0,0,other,2.0,12.443836,32.890411,14.0,1.0,1
2,5008806,1,1,1,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,0,0,0,Security staff,2.0,3.106849,58.832877,30.0,0.0,1
3,5008808,0,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,0,1,1,Sales staff,1.0,8.358904,52.356164,5.0,0.0,1
4,5008809,0,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,0,1,1,Sales staff,1.0,8.358904,52.356164,5.0,0.0,1


In [ ]:
merged_data.drop(columns=['good_debt', 'bad_debt'], inplace=True)

In [ ]:
merged_data.to_csv('merged_data_clean.csv', index=False)